In [1]:
import os
import h5py
import shutil
import zipfile
import requests
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

from sklearn.utils import shuffle
from keras.preprocessing import image
from keras import models,layers,applications


In [ ]:
url_train = 'https://storage.googleapis.com/kaggle-competitions-data/kaggle/5441/train.zip?GoogleAccessId=web-data@kaggle-161607.iam.gserviceaccount.com&Expires=1513750312&Signature=Ubqo2VusO%2FMKiMdoWJytkHwuBi4gG0i%2F0HbXMuJHgz0CqFHZG%2FlM1%2BM3uJ%2Br8RYta4azbsNvKXm4E7k4m2kfuw1juZtzaSTsdJ8GYIM3CuSKyRnFiAidiMGLNy9nHMOhfbDMaOi%2F55uLbcnBcVrhhjwWGv4g4PZqGaSLUX2g7WZb3xcQzI%2BJCskpEgA%2BjY%2FYspaG4yKT%2ByGjMTjsV2br%2Bb83AmbQDD802i5Ng24%2ByDHXTKS5bVjKAE9rYP%2FQZNrefLDaWxpzd3lgqg3X3wuy0ER9MoS6kSKNZfpbQ6IVoRcp6PZ2MrGAtlvSjoZRodkzWX9sT1ip%2B4yxB4yhDS4urQ%3D%3D'
url_test = 'https://storage.googleapis.com/kaggle-competitions-data/kaggle/5441/test.zip?GoogleAccessId=web-data@kaggle-161607.iam.gserviceaccount.com&Expires=1513750320&Signature=qoc1ZpqTtGDESX8O047XBFpcxVrFKVFWDFmdclXIxlZ3rkAaIzpsVMHHbxT5k%2FQlHREs5QWIVnekip3lsL6p%2FHy3wRa%2B39M3e10NyAb7DcJfKyAXAPT2aFp2qU4xDnywriP9NBLYs1xqBch6lidcP9OycrV9QxzWsvF4U4r06Vosi9UhPZ6p8eXOPJUkjBxYGkLc2vD%2F2He2UBkVHOCxVlwMf6aOW4T2XWq%2BM4hnQ7lhKr7g5joAhclx0ZxUudFf3UoJ2eOx3jD1rjNIqPhPRX0eOL99%2BIwX%2Bl7tnGszMT901yVBIuV0gz%2FTrUgkchZv7N6jp5WmALcXP15jYWIPyQ%3D%3D'

train_path = "train.zip"
test_path = "test.zip"

def Download_data(url,path):
    s = requests.Session()
    file = s.get(url)
    with open(path,'wb') as code:
        code.write(file.content)
    
#解压代码
def Un_zip(file_name):  
    """unzip zip file"""  
    zip_file = zipfile.ZipFile(file_name)  
    for names in zip_file.namelist():  
        zip_file.extract(names)  
    zip_file.close() 

Download_data(url_train,train_path)
Download_data(url_test,test_path)
print("下载完成！")

Un_zip(train_path)
Un_zip(test_path)
print("解压完成！")

In [5]:
train_filenames = os.listdir('train')
test_filenames = os.listdir('test')
train_cat = filter(lambda x:x[:3] == 'cat', train_filenames)
train_dog = filter(lambda x:x[:3] == 'dog', train_filenames)

def create_mkdir(dirname):
    if os.path.exists(dirname):
        shutil.rmtree(dirname)
    os.mkdir(dirname)

def file_move(data,old_dir,new_dir):
    for filename in data:
        shutil.move(old_dir+filename, new_dir+filename)   
        
cat_path = 'train2/cat/'
dog_path = 'train2/dog/'

create_mkdir('train2')
os.mkdir(cat_path)
os.mkdir(dog_path)

create_mkdir('test2')
os.mkdir('test2/test')

file_move(train_cat,'train/',cat_path)
file_move(train_dog,'train/',dog_path)
file_move(test_filenames,'test/','test2/test/') 
    
shutil.rmtree("test")
shutil.rmtree("train")
print("分类完成！")

PermissionError: [WinError 5] 拒绝访问。: 'train'

In [ ]:
#显示猫和狗的图像
file_cat = os.listdir(cat_path)
files_dog = os.listdir(dog_path)
for i in range(3):
    image_cat = mpimg.imread(cat_path+file_cat[np.random.randint(0,cats_num)])
    image_dog = mpimg.imread(cat_path+files_dog[np.random.randint(0,dogs_num)])

    plt.subplot(1,2,1)
    plt.title('Cat')
    plt.imshow(image_cat)
    plt.axis('off')
    plt.subplot(1,2,2)
    plt.title('Dog')
    plt.imshow(image_dog)
    plt.axis('off')
    plt.show()

In [ ]:
input_tensor = layers.Input((299, 299, 3))
x = input_tensor
x = applications.inception_v3.preprocess_input(x)

base_model = applications.InceptionV3(input_tensor=x, weights='imagenet', include_top=False)
model = models.Model(base_model.input, layers.GlobalAveragePooling2D()(base_model.output))

generator = image.ImageDataGenerator()
train_generator = generator.flow_from_directory("train2", (299,299), shuffle=False, 
                                          batch_size=16)
test_generator = generator.flow_from_directory("test2", (299,299), shuffle=False, 
                                         batch_size=16, class_mode=None)

train = model.predict_generator(train_generator, train_generator.nb_sample)
test = model.predict_generator(test_generator, test_generator.nb_sample)
with h5py.File("InceptionV3.h5".func_name) as h:
    h.create_dataset("train", data=train)
    h.create_dataset("test", data=test)
    h.create_dataset("label", data=train_generator.classes)

Found 25000 images belonging to 2 classes.
Found 12500 images belonging to 1 classes.


In [ ]:
with h5py.File("InceptionV3.h5", 'r') as h:
    X_train = np.array(h['train'])
    X_test = np.array(h['test'])
    y_train = np.array(h['label'])

X_train, y_train = shuffle(X_train, y_train)

In [ ]:
input_tensor = layers.Input((299, 299, 3))
x = input_tensor
x = applications.inception_v3.preprocess_input(x)

base_model = applications.InceptionV3(input_tensor=x, weights='imagenet', include_top=False)
model = models.Model(base_model.input, layers.GlobalAveragePooling2D()(base_model.output))

generator = image.ImageDataGenerator()
train_generator = generator.flow_from_directory("train2", (299,299), shuffle=False, 
                                          batch_size=16)
test_generator = generator.flow_from_directory("test2", (299,299), shuffle=False, 
                                         batch_size=16, class_mode=None)

X_train = model.predict_generator(train_generator, train_generator.nb_sample)
X_test = model.predict_generator(test_generator, test_generator.nb_sample)
y_train=train_generator.classes

X_train, y_train = shuffle(X_train, y_train)

In [ ]:
input_tensor = layers.Input(X_train.shape)
x = layers.Dropout(0.5)(input_tensor)
x = layers.Dense(1, activation='sigmoid')(x)
model = models.Model(input_tensor, x)

model.compile(optimizer='adadelta',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [ ]:
model.fit(X_train, y_train, batch_size=128, nb_epoch=8, validation_split=0.2)